```{=typst}
#set text(
  font: ("Times New Roman", "LXGW WenKai"),
  size: 11pt,
)

= 问题

#h(2em) （易拉罐的下料）某公司采用一套冲压设备生产一种罐装饮料的易拉罐，这种易拉罐是用镀锡板冲压制成的。易拉罐为圆柱形，包括罐身、上盖和下底，罐身高 10cm，上盖和下底的直径均为 5cm。该公司使用两种不同规格的镀锡板原料，规格 1 的镀锡板为正方形，边长 24cm；规格 2 的镀锡板为长方形，长、宽分别为 32cm 和 28cm。由于生产设备和生产工艺的限制，对于规格 1 的镀锡板原料，只可以按照图 10.7 中的模式 1，2 或 3 进行冲压；对于规格 2 的镀锡板原料只能按照模式 4 进行冲压。使用模式 1，2，3，4 进行每次冲压所需要的时间分别为 1.5s，2s，1s，3s。

#image("Chapter7_3_1.png")

#h(2em) 该工厂每周工作 40h，每周可供使用的规格 1，2 的镀锡板原料分别为 5 万张和 2 万张，目前每只易拉罐的利润为 0.10 元，原料余料损失为 0.001元/cm²（如果周末有罐身、上盖或下底不能配套组装成易拉罐出售，也看作是原料余料损失）。问工厂应如何安排每周的生产？

= 思路

对于盖和底，$S_1 = (pi d^2) / 4 approx 19.6 "cm²"$

对于罐身，$S_2 = pi d h approx 157.1 "cm²"$

#table(
  columns: (1fr, 1fr, 1fr, 1fr, 1fr),
  stroke: none,
  align: center,
  table.hline(),
  table.header([模式], [罐身], [盖底], [损失/cm²], [时间/s]),
  table.hline(stroke: 0.8pt),
  [模式1],
  [1],
  [10],
  [222.6],
  [1.5],
  [模式2],
  [2],
  [4],
  [183.3],
  [2],
  [模式3],
  [0],
  [16],
  [261.8],
  [1],
  [模式4],
  [4],
  [5],
  [169.5],
  [3],
  table.hline(),
)

#h(2em) 设模式 1，2，3，4 使用的原料依次为 $x_1, x_2, x_3, x_4$ 万张；$y_1$ 表示一周生产易拉罐的个数。计算不能配套组装的罐身和底、盖造成的原料损失，用 $y_2$ 表示不配套的罐身个数，$y_3$ 表示不配套的底、盖个数。计量单位均以万计，将值看作实数处理。

#h(2em) 即求解：

$max(0.1 y_1 - 0.001 (222.6 x_1 + 183.3 x_2 + 261.8 x_3 + 169.5 x_4 + 157.1 y_2 + 19.6 y_3))$

S.T.

$
  1.5 x_1 + 2 x_2 + x_3 + 3 x_4 &<= (40 times 3600) / 10^4 = 14.4 \
  x_1 + x_2 + x_3 &<= 5 \
  x_4 &<= 2 \
  y_1 &<= x_1 + 2 x_2 + 4 x_4 \
  y_1 &<= (9x_1 + 3 x_2 + 12 x_3 + 4 x_4) / 2 \
  y_2 &= x_1 + 2 x_2 + 4 x_4 - y_1 \
  y_3 &= 10 x_1 + 4 x_2 + 16 x_3 + 5 x_4 - 2 y_1 \
$

#h(2em) 下面是代码实现：

```


In [ ]:
import numpy as np
import scipy.optimize as opt

In [ ]:
# prepare data
d = 5  # cm
h = 10  # cm
a1 = 24  # cm (= b1)
a2 = 32  # cm
b2 = 28  # cm

coff0 = [1.5, 2, 1, 3]
coff1 = [10, 4, 16, 5]
coff2 = [1, 2, 0, 4]

S1 = np.pi * d * d / 4  # cm^2
print('S1:', S1)
S2 = np.pi * d * h  # cm^2
print('S2:', S2)

dp = [
  a1 * a1 - 10 * S1 - S2,
  a1 * a1 - 4 * S1 - 2 * S2,
  a1 * a1 - 16 * S1,
  a2 * b2 - 5 * S1 - 4 * S2,
]

print('dp:', dp)

In [ ]:
# 定义相关函数
def totalTime(a):  # 10^4 s
  return coff0[0] * a[0] + coff0[1] * a[1] + coff0[2] * a[2] + coff0[3] * a[3]


def y1(a):  # 10^4
  return min(
    # use coff2
    coff2[0] * a[0] + coff2[1] * a[1] + coff2[2] * a[2] + coff2[3] * a[3],
    # use coff1
    (coff1[0] * a[0] + coff1[1] * a[1] + coff1[2] * a[2] + coff1[3] * a[3]) / 2,
  )


def y2(a):  # 10^4, use coff2
  return coff2[0] * a[0] + coff2[1] * a[1] + coff2[2] * a[2] + coff2[3] * a[3] - y1(a)


def y3(a):  # 10^4, use coff1
  return coff1[0] * a[0] + coff1[1] * a[1] + coff1[2] * a[2] + coff1[3] * a[3] - 2 * y1(a)


def target(a):  # 取反，求最小值
  return -0.1 * y1(a) + 0.001 * (dp[0] * a[0] + dp[1] * a[1] + dp[2] * a[2] + dp[3] * a[3] + S2 * y2(a) + S1 * y3(a))

In [ ]:
# 定义约束条件
nlc0 = opt.NonlinearConstraint(totalTime, 0, 14.4)
nlc1 = opt.NonlinearConstraint(y2, 0, np.inf)
nlc2 = opt.NonlinearConstraint(y3, 0, np.inf)

nlc3 = opt.NonlinearConstraint(lambda a: sum(a[0:3]), 0, 5)
nlc4 = opt.NonlinearConstraint(lambda a: a[0:3], np.zeros(3), np.ones(3) * np.inf)
nlc5 = opt.NonlinearConstraint(lambda a: a[3], 0, 2)

result = opt.minimize(target, np.ones(4), constraints=[nlc0, nlc1, nlc2, nlc3, nlc4, nlc5])

print(result)

```{=typst}

= 结论

#h(2em) 0.62 万张用于模式一，4.254 万张用于模式二，0.0 万张用于模式三，1.654 万张用于模式四，利润为 0.3762 万元。

```
